# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [ ]:
import ib_insync
print(ib_insync.__all__)

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [ ]:
from ib_insync import *
util.startLoop()

*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [ ]:
#enable before logging see end of this file
# connect paper
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=37)

2025-04-07 21:39:54,895 ib_insync.wrapper INFO updatePortfolio: PortfolioItem(contract=Stock(conId=4347, symbol='ALB', right='0', primaryExchange='NYSE', currency='USD', localSymbol='ALB', tradingClass='ALB'), position=100.0, marketPrice=58.02152635, marketValue=5802.15, averageCost=73.137174, unrealizedPNL=-1511.56, realizedPNL=0.0, account='DU5417029')
2025-04-07 21:39:54,899 ib_insync.wrapper INFO updatePortfolio: PortfolioItem(contract=Stock(conId=270639, symbol='INTC', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='INTC', tradingClass='NMS'), position=100.0, marketPrice=19.526001, marketValue=1952.6, averageCost=23.348113, unrealizedPNL=-382.21, realizedPNL=0.0, account='DU5417029')
2025-04-07 21:39:54,905 ib_insync.wrapper INFO updatePortfolio: PortfolioItem(contract=Stock(conId=208813720, symbol='GOOG', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='GOOG', tradingClass='NMS'), position=-200.0, marketPrice=149.28697205, marketValue=-29857.39,

If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [ ]:
ib.positions()

# try out deep information

In [ ]:
positions = ib.positions()

In [21]:
for position in positions:
        contract = position.contract
        # market_data = ib.reqMktData(contract)
        ib.reqMktData(contract)
        market_data  = ib.qualifyContracts(contract)
        print("Market Data {}".format(market_data))
        ib.sleep(1)  # Allow time for market data to be received
        ib.reqContractDetails(contract)

        # FROM HERE
        # https://blog.stackademic.com/streaming-real-time-market-data-with-interactive-brokers-tws-api-in-python-a-practical-guide-9fb055fbecc8

        # Request market data for the contract
        market_data = ib.reqMktData(contract)

        if market_data:
                print(f"Bid: {market_data.bid}, Ask: {market_data.ask}, Last: {market_data.last}")
        else:
                print("No market data received.")
        # print("Symbol => {} ".format(contract.localSymbol))
        # print("Symbol => {} ".format(contract.localSymbol))
        # print("Symbol => {} ".format(contract.localSymbol))


Market Data [Option(conId=763248479, symbol='WSM', lastTradeDateOrContractMonth='20250417', strike=150.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='WSM   250417P00150000', tradingClass='WSM')]
Bid: nan, Ask: nan, Last: nan
Market Data [Option(conId=755327783, symbol='BIDU', lastTradeDateOrContractMonth='20250516', strike=95.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='BIDU  250516P00095000', tradingClass='BIDU')]
Bid: nan, Ask: nan, Last: nan
Market Data [Option(conId=755327744, symbol='BIDU', lastTradeDateOrContractMonth='20250516', strike=90.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='BIDU  250516P00090000', tradingClass='BIDU')]
Bid: nan, Ask: nan, Last: nan
Market Data [Stock(conId=270639, symbol='INTC', exchange='ISLAND', primaryExchange='ISLAND', currency='USD', localSymbol='INTC', tradingClass='NMS')]
Bid: nan, Ask: nan, Last: nan
Market Data [Stock(conId=208813720, symbol='GOO

Or filter the account values to get the liquidation value:

In [ ]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [ ]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [ ]:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqContractDetails(contract)

### Current state vs request

Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So it is preferable to use the current state methods over requests. For example, use ``ib.openOrders()`` in preference over ``ib.reqOpenOrders()``, or ``ib.positions()`` over ``ib.reqPositions()``, etc:

In [ ]:
%time l = ib.positions()

In [ ]:
%time l = ib.reqPositions()

### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [ ]:
util.logToConsole()

To see all debug messages (including network traffic):

In [ ]:
import logging
util.logToConsole(logging.DEBUG)



In [ ]:
import logging
util.logToConsole(logging.INFO)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [ ]:
ib.disconnect()